In [ ]:
"""
Example Usage of Optimized Gold Price Predictor
==============================================

This script demonstrates how to use the GoldPricePredictor class
for different scenarios including training, prediction, and analysis.
"""

import pandas as pd
import numpy as np
from gold_price_prediction_optimized import GoldPricePredictor
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def create_sample_data():
    """
    Create sample gold price data for demonstration purposes.
    
    Returns:
        pd.DataFrame: Sample dataset
    """
    np.random.seed(42)
    n_samples = 1000
    
    # Generate sample data
    dates = pd.date_range('2020-01-01', periods=n_samples, freq='D')
    
    # Generate correlated features
    base_trend = np.linspace(1.0, 1.5, n_samples) + np.random.normal(0, 0.02, n_samples)
    
    data = {
        'Date': dates,
        'SPX': 3000 + np.cumsum(np.random.normal(0, 10, n_samples)),  # S&P 500
        'GLD': 150 + np.cumsum(np.random.normal(0, 2, n_samples)),    # Gold ETF
        'USO': 50 + np.cumsum(np.random.normal(0, 1, n_samples)),     # Oil ETF
        'SLV': 20 + np.cumsum(np.random.normal(0, 0.5, n_samples)),   # Silver ETF
        'EUR/USD': base_trend + np.random.normal(0, 0.01, n_samples)  # Target variable
    }
    
    df = pd.DataFrame(data)
    
    # Save sample data
    df.to_csv('sample_gold_price_data.csv', index=False)
    logger.info("Sample data created and saved to 'sample_gold_price_data.csv'")
    
    return df


def example_basic_usage():
    """
    Example of basic usage of the GoldPricePredictor.
    """
    logger.info("=== Basic Usage Example ===")
    
    # Initialize predictor with sample data
    predictor = GoldPricePredictor("sample_gold_price_data.csv")
    
    # Load and explore data
    predictor.load_data()
    predictor.explore_data()
    
    # Create correlation heatmap
    predictor.create_correlation_heatmap()
    
    # Preprocess data
    predictor.dataset = predictor.preprocess_data()
    
    # Prepare features
    X, y = predictor.prepare_features()
    
    # Split and scale data
    predictor.split_and_scale_data(X, y)
    
    # Train a single model (XGBoost)
    results = predictor.train_xgboost_model()
    
    logger.info(f"XGBoost Model Results:")
    logger.info(f"  Train R²: {results['train_r2']:.4f}")
    logger.info(f"  Test R²: {results['test_r2']:.4f}")
    logger.info(f"  Test RMSE: {results['test_rmse']:.4f}")
    
    # Plot feature importance
    predictor.plot_feature_importance('xgboost')
    
    # Save model
    predictor.save_best_model('xgboost', 'example_model.pkl')


def example_comprehensive_analysis():
    """
    Example of comprehensive analysis with all models.
    """
    logger.info("=== Comprehensive Analysis Example ===")
    
    # Initialize predictor
    predictor = GoldPricePredictor("sample_gold_price_data.csv")
    
    # Complete pipeline
    predictor.load_data()
    predictor.dataset = predictor.preprocess_data()
    X, y = predictor.prepare_features()
    predictor.split_and_scale_data(X, y)
    
    # Train all models
    predictor.train_lasso_model()
    predictor.train_random_forest_model()
    predictor.train_xgboost_model()
    
    # Compare models
    comparison = predictor.compare_models()
    print("\nModel Comparison:")
    print(comparison.to_string(index=False))
    
    # Create visualizations
    predictor.create_visualizations()
    
    # Plot feature importance for all tree-based models
    predictor.plot_feature_importance('random_forest')
    predictor.plot_feature_importance('xgboost')


def example_prediction():
    """
    Example of making predictions with a trained model.
    """
    logger.info("=== Prediction Example ===")
    
    # Initialize and train model
    predictor = GoldPricePredictor("sample_gold_price_data.csv")
    predictor.load_data()
    predictor.dataset = predictor.preprocess_data()
    X, y = predictor.prepare_features()
    predictor.split_and_scale_data(X, y)
    predictor.train_xgboost_model()
    
    # Create new data for prediction
    new_data = pd.DataFrame({
        'SPX': [3500, 3600, 3400],
        'GLD': [180, 185, 175],
        'USO': [55, 60, 50],
        'price_trend': [1.25, 1.30, 1.20]
    })
    
    # Make predictions
    predictions = predictor.predict(new_data, model_name='xgboost')
    
    logger.info("Predictions for new data:")
    for i, pred in enumerate(predictions):
        logger.info(f"  Sample {i+1}: {pred:.4f}")


def example_custom_analysis():
    """
    Example of custom analysis with different parameters.
    """
    logger.info("=== Custom Analysis Example ===")
    
    # Initialize predictor
    predictor = GoldPricePredictor("sample_gold_price_data.csv")
    predictor.load_data()
    
    # Custom preprocessing
    predictor.dataset = predictor.preprocess_data(drop_slv=False)  # Keep SLV
    
    # Custom feature preparation
    X, y = predictor.prepare_features(target_column='EUR/USD')
    
    # Custom train/test split
    predictor.split_and_scale_data(X, y, test_size=0.3, random_state=123)
    
    # Train only Random Forest with custom parameters
    results = predictor.train_random_forest_model()
    
    logger.info("Custom Analysis Results:")
    logger.info(f"  Test size: 30%")
    logger.info(f"  Random Forest Test R²: {results['test_r2']:.4f}")
    logger.info(f"  Best parameters: {results['best_params']}")


def main():
    """
    Main function to run all examples.
    """
    try:
        # Create sample data if it doesn't exist
        try:
            pd.read_csv("sample_gold_price_data.csv")
            logger.info("Sample data already exists")
        except FileNotFoundError:
            create_sample_data()
        
        # Run examples
        example_basic_usage()
        print("\n" + "="*50 + "\n")
        
        example_comprehensive_analysis()
        print("\n" + "="*50 + "\n")
        
        example_prediction()
        print("\n" + "="*50 + "\n")
        
        example_custom_analysis()
        
        logger.info("All examples completed successfully!")
        
    except Exception as e:
        logger.error(f"Error running examples: {str(e)}")
        raise


if __name__ == "__main__":
    main()
